# Accessibilité relative des résidus et évolution protéique

Il y a plusieurs étapes dans le traitement des données :

 * obtenir les données pdb par **PISCES**
 * faire tourner **DSSP** pour obtenir l'accessibilité du solvant
 * récupération des séquences homologues par **PSI-BLAST** (pas moins de 70% d'identité)
 * réalignement avec la séquence parente
 * repositionnement des chaines latérales avec **SCWRL4** -> accessibilité du solvant ? 

## PISCES

Sélection de la base de données la plus petite base de [données](http://dunbrack.fccc.edu/Guoli/culledpdb_hh/cullpdb_pc20_res1.6_R0.25_d201103_chains3760.gz)

*Afin de simplifier le traitement on travaille tout d'abord avec un fichier PDB, on résoud les problèmes, une fois que les exceptions sont réglées, on automatise.*

Pour récupérer les fichiers PDB par ligne de commande simplement faire :  
`wget https://files.rcsb.org/download/XXXX.pdb`  
Le fichier pdb se créera dans le dossier sous le nom XXXX.pdb avec XXXX l'ID de la protéine.

In [1]:
def pdbfiles(pisces_file):
    """Creates list of [pdb id, chain, length]
    """
    with open(pisces_file, 'r') as file:
        pdb_list = []
        next(file)
        for line in file:
            line = line.split()
            pdb_list.append([line[0][0:4], line[0][4:], int(line[1])])
        return pdb_list

In [2]:
PDB = pdbfiles('../../cullpdb_pc20_res1.6_R0.25_d201103_chains3760')

On récupère ainsi une liste contenant pour chaque protéine une liste de son identifiant, la chaine choisie et sa taille.

**to check :**
* complete (taille pdb = taille prot dans la liste)
* gerer les differentes chaines
* gerer les grands fichiers

In [3]:
from Bio.PDB import PDBList

p = PDBList()
pdb = p.retrieve_pdb_file(pdb_code = '1AH7', file_format = 'pdb', pdir = '../data/')

Structure exists: '../data/pdb1ah7.ent' 


In [4]:
import os
from Bio import PDB

class ChainSplitter:
    def __init__(self, out_dir=None):
        """ Create parsing and writing objects, specify output directory. """
        self.parser = PDB.PDBParser()
        self.writer = PDB.PDBIO()
        if out_dir is None:
            out_dir = '../data/'
        self.out_dir = out_dir

    def make_pdb(self, pdb_path, chain_letters, overwrite=False, struct=None):
        """ Create a new PDB file containing only the specified chains.

        Returns the path to the created file.

        :param pdb_path: full path to the crystal structure
        :param chain_letters: iterable of chain characters (case insensitive)
        :param overwrite: write over the output file if it exists
        """
        chain_letters = [chain.upper() for chain in chain_letters]

        # Input/output files
        (pdb_dir, pdb_fn) = os.path.split(pdb_path)
        pdb_id = pdb_fn[3:7]
        out_name = "pdb%s_%s.ent" % (pdb_id, "".join(chain_letters))
        out_path = os.path.join(self.out_dir, out_name)
        print "OUT PATH:",out_path
        plural = "s" if (len(chain_letters) > 1) else ""  # for printing

        # Skip PDB generation if the file already exists
        if (not overwrite) and (os.path.isfile(out_path)):
            print("Chain%s %s of '%s' already extracted to '%s'." %
                    (plural, ", ".join(chain_letters), pdb_id, out_name))
            return out_path

        print("Extracting chain%s %s from %s..." % (plural,
                ", ".join(chain_letters), pdb_fn))

        # Get structure, write new file with only given chains
        if struct is None:
            struct = self.parser.get_structure(pdb_id, pdb_path)
        self.writer.set_structure(struct)
        self.writer.save(out_path, select=SelectChains(chain_letters))

        return out_path


class SelectChains(PDB.Select):
    """ Only accept the specified chains when saving. """
    def __init__(self, chain_letters):
        self.chain_letters = chain_letters

    def accept_chain(self, chain):
        return (chain.get_id() in self.chain_letters)


if __name__ == "__main__":
    """ Parses PDB id's desired chains, and creates new PDB structures. """
    import sys
    if not len(sys.argv) == 2:
        print "Usage: $ python %s 'pdb.txt'" % __file__
        sys.exit()

    pdb_textfn = sys.argv[1]

    pdbList = PDB.PDBList()
    splitter = ChainSplitter("/home/steve/chain_pdbs")  # Change me.

    with open(pdb_textfn) as pdb_textfile:
        for line in pdb_textfile:
            pdb_id = line[:4].lower()
            chain = line[4]
            pdb_fn = pdbList.retrieve_pdb_file(pdb_id)
            splitter.make_pdb(pdb_fn, chain)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("OUT PATH:",out_path)? (<ipython-input-4-f41b9e607589>, line 29)

## DSSP

In [4]:
import wget

pdbfiles = wget.download('https://files.rcsb.org/download/1AHO.pdb', out = '../data/1AHO.pdb')
pdbfiles = wget.download('https://files.rcsb.org/download/1BTE.pdb', out = '../data/1BTE.pdb')

### Traitement de 1AHO

In [5]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

p = PDBParser()
structure = p.get_structure("1AHO","../data/1AHO.pdb")
model = structure[0]
DSSP = DSSP(model, "../data/1AHO.pdb")

In [15]:
def full_structure(DSSP_dict):
    """ Check si certains acides aminés n'ont pas de structure secondaire associés
    """
    for key in DSSP_dict.keys():
        if DSSP_dict[key][2] == '-':
            return False
    return True

In [16]:
print(full_structure(DSSP)) #si False -> certains AA n'ont pas de structure secondaire predites

False


In [20]:
ASA = 0
for k in DSSP.keys():
    #print(DSSP[k]) #affichage de chaque element pour tous les AA
    ASA += DSSP[k][3] #recuperation de la 'relative ASA' de chaque AA

print("Accessibilite relative du solvant pour la proteine entiere : {}".format(ASA))

(1, 'E', '-', 0.845360824742268, 360.0, 22.7, 0, 0.0, 79, -0.5, 0, 0.0, 2, -0.5)
(2, 'T', '-', 0.0, -122.8, 117.6, 77, -0.2, 28, -0.1, 1, -0.1, 3, -0.1)
(3, 'Q', '-', 0.4292929292929293, -111.6, -20.0, -2, -0.5, 22, -2.1, 1, -0.3, 2, -0.3)
(4, 'E', 'E', 0.4175257731958763, -153.8, 150.9, 20, -0.3, 77, -2.3, 45, -0.1, 78, -0.3)
(5, 'C', 'E', 0.0, -139.2, 159.2, 18, -2.1, 18, -2.5, -2, -0.3, 2, -0.3)
(6, 'L', 'E', 0.23780487804878048, -65.6, 132.7, -2, -0.3, 2, -0.4, 16, -0.2, 27, -0.2)
(7, 'F', 'E', 0.18274111675126903, -110.4, 133.2, 14, -2.9, 14, -2.0, -2, -0.3, 2, -0.3)
(8, 'F', 'E', 0.25380710659898476, -148.5, 138.3, 40, -2.2, 40, -2.4, -2, -0.4, 2, -0.3)
(9, 'N', 'E', 0.025477707006369428, -145.8, 103.5, 10, -2.3, 3, -1.4, -2, -0.3, 38, -0.2)
(10, 'A', 'T', 0.42452830188679247, -63.8, -21.4, 36, -2.2, 3, -0.3, -2, -0.3, 37, -0.1)
(11, 'N', 'T', 0.33121019108280253, -96.0, 27.4, 35, -0.3, 4, -2.4, 1, -0.2, -1, -0.3)
(12, 'W', 'H', 0.33480176211453744, -55.8, -38.7, -3, -1.4, 5, -1.

In [8]:
# Recuperation de la sequence fasta de la proteine
fasta = ""
for key in DSSP.keys():
    fasta += DSSP[key][1] #recuperation du symbole 1 lettre de l'AA
print(fasta)

VKDGYIVDDVNCTYFCGRNAYCNEECTKLKGESGYCQWASPYGNACYCYKLPDHVRTKGPGRCH


### Traitement de 1BTE

In [17]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

p = PDBParser()
structure = p.get_structure("1BTE","../data/1BTE.pdb")
model = structure[0]
DSSP = DSSP(model, "../data/1BTE.pdb")

In [18]:
ASA = 0
for k in DSSP.keys():
    #print(DSSP[k]) #affichage de chaque element pour tous les AA
    ASA += DSSP[k][3] #recuperation de la 'relative ASA' de chaque AA

print("Accessibilite relative du solvant pour la proteine entiere : {}".format(ASA))

Accessibilite relative du solvant pour la proteine entiere : 34.529479532768015


In [19]:
# Recuperation de la sequence fasta de la proteine
fasta = ""
for key in DSSP.keys():
    fasta += DSSP[key][1] #recuperation du symbole 1 lettre de l'AA
print(fasta)

ETQECLFFNANWERDRTNQTGVEPCYGRRHCFATWKNISGSIEIVKQGCWLDDINCYDRTDCIEKKDSPEVYFCCCEGNMCNEKFSYFPEM


## PSI-BLAST

In [12]:
from Bio.Blast.Applications import NcbipsiblastCommandline

In [13]:
cline = NcbipsiblastCommandline(help)
cline

NcbipsiblastCommandline(cmd=Type help() for interactive help, or help(object) for help about object.)

In [14]:
cline = NcbipsiblastCommandline('psiblast', db = 'refseq_prot', query = fasta)

In [15]:
str(cline)

'psiblast -query XNLTELKNTPVSELITLGENXGLENLARXRKQDIIFAILKQHAKSGEDIFGDGVLEILQDGFGFLRSADSSYLAGPDDIYVSPSQIRRFNLRTGDTISGKIRPPKEGERYFALLKVNEVNFDKPE -db refseq_prot'

In [16]:
cline()

ApplicationError: Non-zero return code 127 from 'psiblast -query XNLTELKNTPVSELITLGENXGLENLARXRKQDIIFAILKQHAKSGEDIFGDGVLEILQDGFGFLRSADSSYLAGPDDIYVSPSQIRRFNLRTGDTISGKIRPPKEGERYFALLKVNEVNFDKPE -db refseq_prot', message '/bin/sh: 1: psiblast: not found'

## ClustalOmega

## SCWRL

## DSSP 

In [22]:
#imports
import wget
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

p = PDBParser()
structure = p.get_structure("5NH3","../data/5NH3.pdb")
model = structure[0]
DSSP = DSSP(model, "../data/5NH3.pdb")

ASA = 0
for k in DSSP.keys():
    #print(DSSP[k]) #affichage de chaque element pour tous les AA
    ASA += DSSP[k][3] #recuperation de la 'relative ASA' de chaque AA

print("Accessibilite relative du solvant pour la proteine entiere : {}".format(ASA))

# Recuperation de la sequence fasta de la proteine
fasta = ""
for key in DSSP.keys():
    fasta += DSSP[key][1] #recuperation du symbole 1 lettre de l'AA
print(fasta)

Accessibilite relative du solvant pour la proteine entiere : 35.50062074974891
SETQECLFFNANWEKDRTNQTGVEPCYGDKDKRRHCFATWKNISGSIEIVKQGCWLDDINCYDRTDCVEKKDSPEVYFCCCEGNMCNEKFSYFP


In [ ]:
p = PDBParser()
structure = p.get_structure("4AEI","../data/4AEI.pdb")
model = structure[0]
DSSP = DSSP(model, "../data/4AEI.pdb")

ASA = 0
for k in DSSP.keys():
    #print(DSSP[k]) #affichage de chaque element pour tous les AA
    ASA += DSSP[k][3] #recuperation de la 'relative ASA' de chaque AA

print("Accessibilite relative du solvant pour la proteine entiere : {}".format(ASA))

## Comparaison